In [ ]:
%matplotlib inline

import pydicom
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from ipywidgets import interact

## Einlesen und Visualisieren eines DICOM-Bildes

In [ ]:
ds = pydicom.dcmread("/path/to/file.dcm")
img = ds.pixel_array
text_data = "Image Height: " + str(img.shape[0]) + "\nImage Width: " + str(img.shape[1]) + "\nPatient Name: " + str(ds.PatientName) + "\nPatient ID: " + ds.PatientID + "\nPatient DOB: " + ds.PatientBirthDate + "\nPatient Sex: " + ds.PatientSex
plt.text(img.shape[1] + 5, img.shape[0] / 2, text_data)
plt.imshow(img, cmap="bone");

## Fenster-Funktion

In [ ]:
def windowFunction(uG, oG, img):
    new_img = np.copy(img)
    new_img = new_img.clip(uG, oG)

In [ ]:
width_slider = widgets.IntSlider(min=2, max=4095, step=1, value=300, continuous_update=False)
center_slider = widgets.IntSlider(min=-1024, max=3071, step=1, value=25, continuous_update=False)

def interact_windowing(width, center):
    center = center / ds.RescaleSlope - ds.RescaleIntercept
    uG = center - (width // 2)
    oG = center + (width // 2)
    plt.axis("off")
    plt.imshow(windowFunction(uG, oG, img), cmap="gray");

interact(interact_windowing, width=width_slider, center=center_slider);

## Binarisierungsfunktion

In [ ]:
def to_binary(img, thresh):
    return (img >= thresh) * 1

In [ ]:
def img_opt(img, box, thresh):
    # box = (75, 450, 125, 380)
    bin_img = to_binary(img, threshold)
    bin_img[:,:box[0]] = 0
    bin_img[:,box[1]:] = 0
    bin_img[:box[2]] = 0
    bin_img[box[3]:] = 0
    bin_img = skimage.morphology.binary_erosion(bin_img)
    bin_img = skimage.morphology.binary_closing(bin_img)
    bin_img = skimage.morphology.binary_dilation(bin_img)
    return bin_img

## 3D-Rekonstruktion

In [ ]:
img_list = [img_opt(img.pixel_array) for img in imgs]
img_array = np.array(img_list)
img_array = img_array.transpose(2, 1, 0)
verts, faces, normals, values = measure.marching_cubes(img_array, 0)
x, y, z = zip(*verts)
mesh = Poly3DCollection(verts[faces], alpha=0.6)
ax = plt.figure(figsize=(10,10)).add_subplot(projection="3d")
ax.add_collection3d(mesh)
ax.set_xlim(0, max(x));
ax.set_ylim(0, max(x));
ax.set_zlim(0, max(x));
ax.set_xlabel("size (mm)");
ax.set_ylabel("size (mm)");
ax.set_zlabel("size (mm)");